### Seminarul 1

In [17]:
import base64

coded_string = "QUVTLTI1NiBFQ0IgbW9kZSB0d2ljZSwgdHdvIGtleXM="
decoded_string = base64.b64decode(coded_string)
print(decoded_string)

b'AES-256 ECB mode twice, two keys'


In [24]:
from Crypto.Cipher import AES
from hashlib import md5
from base64 import b64decode
from base64 import b64encode

unpad = lambda s: s[:-ord(s[len(s) - 1:])]

class AESCipher:
    def __init__(self, key):
        self.key = md5(key.encode('utf-8')).hexdigest()
        
    def decrypt(self, enc):
        enc = b64decode(enc)
        cipher = AES.new(self.key, AES.MODE_ECB)
        return unpad(cipher.decrypt(enc)).decode('utf-8')

In [46]:
from Crypto.Cipher import AES
plain = base64.b64decode('QUVTLTI1NiBFQ0IgbW9kZSB0d2ljZSwgdHdvIGtleXM=')
cipher = base64.b64decode('THbpB4bE82Rq35khemTQ10ntxZ8sf7s2WK8ErwcdDEc=')
unknown = base64.b64decode('s5hd0ThTkv1U44r9aRyUhaX5qJe561MZ16071nlvM9U=')
 
KEY_PADDING = chr(0)*29
 
def NewAES(key):
    return AES.new(key, mode=AES.MODE_ECB)
 
def Encrypt(short_key, text=plain):
    return NewAES(KEY_PADDING+short_key).encrypt(text)
 
def Decrypt(short_key, text=cipher):
    return NewAES(KEY_PADDING+short_key).decrypt(text)
 
def KeyGen():
    """Generator for all possible 24 bit keys."""
    for a in xrange(0, 256):
        for b in xrange(0, 256):
            for c in xrange(0, 256):
                yield chr(a)+chr(b)+chr(c)
                
def EncryptTable():
    """Map of encryptions to keys."""
    table = {}
    for short_key in KeyGen():
        table[Encrypt(short_key)] = short_key
    return table

table = EncryptTable()
for short_key in KeyGen():
    decrypted = Decrypt(short_key)
    if decrypted in table:
    # Have a match, now decrypt the unknown
        print (Decrypt(short_key, Decrypt(table[decrypted], unknown)))
        break

NameError: name 'xrange' is not defined